In [ ]:
import pandas as pd
pd.__version__

from google.colab import drive
drive.mount('/content/drive/')
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
drive  sample_data


# **Primera opción: Primer dato - Último dato**

---



In [ ]:
df= pd.read_excel('/content/drive/Shareddrives/BID/Ciudades/Metadatos/reshape.xlsx')

In [ ]:
# Terminar de rellenar los valores faltantes en las columnas de país y ciudad
nombres= pd.DataFrame()
nombres['Ciudad']= df['Ciudad']
nombres['País']= df['País']
nombres['Region']= df['Region']

In [ ]:
nombres= nombres.ffill()
nombres

,Ciudad,País,Region
0,Bahía Blanca,Argentina,Mercosur
1,Bahía Blanca,Argentina,Mercosur
2,Bahía Blanca,Argentina,Mercosur
3,Bahía Blanca,Argentina,Mercosur
4,Bahía Blanca,Argentina,Mercosur
...,...,...,...
1969,VILLAVICENCIO,Colombia,Andino
1970,VILLAVICENCIO,Colombia,Andino
1971,VILLAVICENCIO,Colombia,Andino
1972,VILLAVICENCIO,Colombia,Andino


In [ ]:
df=df.drop(columns=['País', 'Ciudad', 'Region'])
df['País']= nombres['País']
df['Ciudad']= nombres['Ciudad']
df['Region']= nombres['Region']

In [ ]:
df= df[['País', 'Ciudad','Region','Sector_renombrado', 'Periodo1', 'Periodo2']]
df

,País,Ciudad,Region,Sector_renombrado,Periodo1,Periodo2
0,Argentina,Bahía Blanca,Mercosur,Actividades Financieras,1009.000000,1262.00000
1,Argentina,Bahía Blanca,Mercosur,Agricultura,1360.000000,2119.00000
2,Argentina,Bahía Blanca,Mercosur,Alojamiento,3123.000000,4416.00000
3,Argentina,Bahía Blanca,Mercosur,Comercio,30746.000000,25440.00000
4,Argentina,Bahía Blanca,Mercosur,Construcción,13106.000000,11173.00000
...,...,...,...,...,...,...
1969,Colombia,VILLAVICENCIO,Andino,Construcción,18941.389890,22331.32029
1970,Colombia,VILLAVICENCIO,Andino,Manufactura,14201.411780,15920.80940
1971,Colombia,VILLAVICENCIO,Andino,Servicios,98900.003710,93135.34375
1972,Colombia,VILLAVICENCIO,Andino,Telecomunicaciones,2477.745566,3323.70907


In [ ]:
# Se eliminan los valores de totales nacionales para evitar doble contabilidad
df = df.drop(df[df['Ciudad']=='Total Nacional'].index)
df = df.drop(df[df['Ciudad']=='Total Nacional Urbano'].index)
df = df.drop(df[df['Ciudad']=='Total Nacional'].index)
df = df.drop(df[df['Sector_renombrado']=='Total'].index)
df = df.drop(df[df['Sector_renombrado']=='NS/NR'].index)

In [ ]:
#Se convierten en 0 los missing values
df=df.fillna(0)

In [ ]:
#Se calcula el total de empleo por país y actividad económica
newcolumn= df.groupby(by=['País', 'Sector_renombrado']).sum()
newcolumn= newcolumn.rename(columns={'Periodo1':'Total1', 'Periodo2':'Total2'})
newcolumn

Total1        Total2
País      Sector_renombrado                               
Argentina Actividades Financieras   205170.0  2.544680e+05
          Agricultura               102372.0  1.230160e+05
          Alojamiento               455911.0  4.063220e+05
          Comercio                 2090782.0  2.117365e+06
          Construcción             1041403.0  1.022848e+06
...                                      ...           ...
Uruguay   Construcción              114826.0  1.074514e+05
          Manufactura               149772.0  1.434726e+05
          Servicios                 680685.0  7.012809e+05
          Telecomunicaciones         33925.0  4.267217e+04
          Transporte                 67244.0  8.248162e+04

[137 rows x 2 columns]

In [ ]:
#Se adicionan los totales a la base
df= df.join(newcolumn, on=['País', 'Sector_renombrado'])

In [ ]:
# Corregir valores 0 en el periodo 1
df['Periodo1_0']= df['Periodo1'] 
df.loc[df['Periodo1'] == 0, 'Periodo1_0'] = 1

In [ ]:
df

,País,Ciudad,Region,Sector_renombrado,Periodo1,Periodo2,Total1,Total2,Periodo1_0
0,Argentina,Bahía Blanca,Mercosur,Actividades Financieras,1009.000000,1262.00000,2.051700e+05,2.544680e+05,1009.000000
1,Argentina,Bahía Blanca,Mercosur,Agricultura,1360.000000,2119.00000,1.023720e+05,1.230160e+05,1360.000000
2,Argentina,Bahía Blanca,Mercosur,Alojamiento,3123.000000,4416.00000,4.559110e+05,4.063220e+05,3123.000000
3,Argentina,Bahía Blanca,Mercosur,Comercio,30746.000000,25440.00000,2.090782e+06,2.117365e+06,30746.000000
4,Argentina,Bahía Blanca,Mercosur,Construcción,13106.000000,11173.00000,1.041403e+06,1.022848e+06,13106.000000
...,...,...,...,...,...,...,...,...,...
1969,Colombia,VILLAVICENCIO,Andino,Construcción,18941.389890,22331.32029,1.222907e+06,1.287023e+06,18941.389890
1970,Colombia,VILLAVICENCIO,Andino,Manufactura,14201.411780,15920.80940,2.196661e+06,1.848735e+06,14201.411780
1971,Colombia,VILLAVICENCIO,Andino,Servicios,98900.003710,93135.34375,7.137919e+06,6.662537e+06,98900.003710
1972,Colombia,VILLAVICENCIO,Andino,Telecomunicaciones,2477.745566,3323.70907,2.731488e+05,3.198908e+05,2477.745566


In [ ]:
#Se calcula el total de ocupación por país 
newcolumn_1=pd.DataFrame()
newcolumn_1= newcolumn.groupby(by='País').sum()
newcolumn_1

,Total1,Total2
País,,
Argentina,1.092180e+07,1.141847e+07
Bahamas,1.754900e+05,1.902300e+05
Bolivia,3.146365e+06,4.012502e+06
Brasil,9.124434e+07,9.292227e+07
Chile,6.178856e+05,7.239959e+05
Colombia,1.777735e+07,1.620882e+07
Costa Rica,2.110799e+06,2.076832e+06
Ecuador,6.562889e+06,6.338886e+06
El Salvador,1.330221e+06,2.838840e+06


# **Para el National Component**

In [ ]:
#Crecimiento anual 
newcolumn_1['ns']= (newcolumn_1.Total2- newcolumn_1.Total1)/newcolumn_1.Total1
newcolumn_1= newcolumn_1.drop(columns=['Total1', 'Total2'])

In [ ]:
df= df.join(newcolumn_1, on=['País'])

In [ ]:
df['ns_labor'] = df['Periodo1_0']*df['ns']

In [ ]:
df

,País,Ciudad,Region,Sector_renombrado,Periodo1,Periodo2,Total1,Total2,Periodo1_0,ns,ns_labor
0,Argentina,Bahía Blanca,Mercosur,Actividades Financieras,1009.000000,1262.00000,2.051700e+05,2.544680e+05,1009.000000,0.045475,45.884108
1,Argentina,Bahía Blanca,Mercosur,Agricultura,1360.000000,2119.00000,1.023720e+05,1.230160e+05,1360.000000,0.045475,61.845775
2,Argentina,Bahía Blanca,Mercosur,Alojamiento,3123.000000,4416.00000,4.559110e+05,4.063220e+05,3123.000000,0.045475,142.017908
3,Argentina,Bahía Blanca,Mercosur,Comercio,30746.000000,25440.00000,2.090782e+06,2.117365e+06,30746.000000,0.045475,1398.169256
4,Argentina,Bahía Blanca,Mercosur,Construcción,13106.000000,11173.00000,1.041403e+06,1.022848e+06,13106.000000,0.045475,595.993179
...,...,...,...,...,...,...,...,...,...,...,...
1969,Colombia,VILLAVICENCIO,Andino,Construcción,18941.389890,22331.32029,1.222907e+06,1.287023e+06,18941.389890,-0.088232,-1671.240972
1970,Colombia,VILLAVICENCIO,Andino,Manufactura,14201.411780,15920.80940,2.196661e+06,1.848735e+06,14201.411780,-0.088232,-1253.022158
1971,Colombia,VILLAVICENCIO,Andino,Servicios,98900.003710,93135.34375,7.137919e+06,6.662537e+06,98900.003710,-0.088232,-8726.167368
1972,Colombia,VILLAVICENCIO,Andino,Telecomunicaciones,2477.745566,3323.70907,2.731488e+05,3.198908e+05,2477.745566,-0.088232,-218.617004


# **Para el Industry mix component**

In [ ]:
df['im_labor']=df['Periodo1_0']*((df['Total2']-df['Total1'])/df['Total1']-df['ns'])

In [ ]:
df

,País,Ciudad,Region,Sector_renombrado,Periodo1,Periodo2,Total1,Total2,Periodo1_0,ns,ns_labor,im_labor
0,Argentina,Bahía Blanca,Mercosur,Actividades Financieras,1009.000000,1262.00000,2.051700e+05,2.544680e+05,1009.000000,0.045475,45.884108,196.557195
1,Argentina,Bahía Blanca,Mercosur,Agricultura,1360.000000,2119.00000,1.023720e+05,1.230160e+05,1360.000000,0.045475,61.845775,212.407341
2,Argentina,Bahía Blanca,Mercosur,Alojamiento,3123.000000,4416.00000,4.559110e+05,4.063220e+05,3123.000000,0.045475,142.017908,-481.703607
3,Argentina,Bahía Blanca,Mercosur,Comercio,30746.000000,25440.00000,2.090782e+06,2.117365e+06,30746.000000,0.045475,1398.169256,-1007.252882
4,Argentina,Bahía Blanca,Mercosur,Construcción,13106.000000,11173.00000,1.041403e+06,1.022848e+06,13106.000000,0.045475,595.993179,-829.506842
...,...,...,...,...,...,...,...,...,...,...,...,...
1969,Colombia,VILLAVICENCIO,Andino,Construcción,18941.389890,22331.32029,1.222907e+06,1.287023e+06,18941.389890,-0.088232,-1671.240972,2664.329763
1970,Colombia,VILLAVICENCIO,Andino,Manufactura,14201.411780,15920.80940,2.196661e+06,1.848735e+06,14201.411780,-0.088232,-1253.022158,-996.323566
1971,Colombia,VILLAVICENCIO,Andino,Servicios,98900.003710,93135.34375,7.137919e+06,6.662537e+06,98900.003710,-0.088232,-8726.167368,2139.466547
1972,Colombia,VILLAVICENCIO,Andino,Telecomunicaciones,2477.745566,3323.70907,2.731488e+05,3.198908e+05,2477.745566,-0.088232,-218.617004,642.616070


# **Para el regional Shift component**

In [ ]:
df['rs_labor']= df['Periodo1_0']*(((df['Periodo2']-df['Periodo1'])/df['Periodo1_0'])-((df['Total2']-df['Total1'])/df['Total1'])) 

In [ ]:
df

,País,Ciudad,Region,Sector_renombrado,Periodo1,Periodo2,Total1,Total2,Periodo1_0,ns,ns_labor,im_labor,rs_labor
0,Argentina,Bahía Blanca,Mercosur,Actividades Financieras,1009.000000,1262.00000,2.051700e+05,2.544680e+05,1009.000000,0.045475,45.884108,196.557195,10.558698
1,Argentina,Bahía Blanca,Mercosur,Agricultura,1360.000000,2119.00000,1.023720e+05,1.230160e+05,1360.000000,0.045475,61.845775,212.407341,484.746884
2,Argentina,Bahía Blanca,Mercosur,Alojamiento,3123.000000,4416.00000,4.559110e+05,4.063220e+05,3123.000000,0.045475,142.017908,-481.703607,1632.685700
3,Argentina,Bahía Blanca,Mercosur,Comercio,30746.000000,25440.00000,2.090782e+06,2.117365e+06,30746.000000,0.045475,1398.169256,-1007.252882,-5696.916374
4,Argentina,Bahía Blanca,Mercosur,Construcción,13106.000000,11173.00000,1.041403e+06,1.022848e+06,13106.000000,0.045475,595.993179,-829.506842,-1699.486336
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,Colombia,VILLAVICENCIO,Andino,Construcción,18941.389890,22331.32029,1.222907e+06,1.287023e+06,18941.389890,-0.088232,-1671.240972,2664.329763,2396.841609
1970,Colombia,VILLAVICENCIO,Andino,Manufactura,14201.411780,15920.80940,2.196661e+06,1.848735e+06,14201.411780,-0.088232,-1253.022158,-996.323566,3968.743344
1971,Colombia,VILLAVICENCIO,Andino,Servicios,98900.003710,93135.34375,7.137919e+06,6.662537e+06,98900.003710,-0.088232,-8726.167368,2139.466547,822.040862
1972,Colombia,VILLAVICENCIO,Andino,Telecomunicaciones,2477.745566,3323.70907,2.731488e+05,3.198908e+05,2477.745566,-0.088232,-218.617004,642.616070,421.964438


# **Cambio total en el empleo**


In [ ]:
df['c_labor']= df['ns_labor']+df['rs_labor']+df['im_labor']

In [ ]:
#df=df.drop(columns=['ns'])
df

,País,Ciudad,Region,Sector_renombrado,Periodo1,Periodo2,Total1,Total2,Periodo1_0,ns,ns_labor,im_labor,rs_labor,c_labor
0,Argentina,Bahía Blanca,Mercosur,Actividades Financieras,1009.000000,1262.00000,2.051700e+05,2.544680e+05,1009.000000,0.045475,45.884108,196.557195,10.558698,253.000000
1,Argentina,Bahía Blanca,Mercosur,Agricultura,1360.000000,2119.00000,1.023720e+05,1.230160e+05,1360.000000,0.045475,61.845775,212.407341,484.746884,759.000000
2,Argentina,Bahía Blanca,Mercosur,Alojamiento,3123.000000,4416.00000,4.559110e+05,4.063220e+05,3123.000000,0.045475,142.017908,-481.703607,1632.685700,1293.000000
3,Argentina,Bahía Blanca,Mercosur,Comercio,30746.000000,25440.00000,2.090782e+06,2.117365e+06,30746.000000,0.045475,1398.169256,-1007.252882,-5696.916374,-5306.000000
4,Argentina,Bahía Blanca,Mercosur,Construcción,13106.000000,11173.00000,1.041403e+06,1.022848e+06,13106.000000,0.045475,595.993179,-829.506842,-1699.486336,-1933.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1969,Colombia,VILLAVICENCIO,Andino,Construcción,18941.389890,22331.32029,1.222907e+06,1.287023e+06,18941.389890,-0.088232,-1671.240972,2664.329763,2396.841609,3389.930400
1970,Colombia,VILLAVICENCIO,Andino,Manufactura,14201.411780,15920.80940,2.196661e+06,1.848735e+06,14201.411780,-0.088232,-1253.022158,-996.323566,3968.743344,1719.397620
1971,Colombia,VILLAVICENCIO,Andino,Servicios,98900.003710,93135.34375,7.137919e+06,6.662537e+06,98900.003710,-0.088232,-8726.167368,2139.466547,822.040862,-5764.659960
1972,Colombia,VILLAVICENCIO,Andino,Telecomunicaciones,2477.745566,3323.70907,2.731488e+05,3.198908e+05,2477.745566,-0.088232,-218.617004,642.616070,421.964438,845.963504


In [ ]:
df.to_csv('/content/drive/MyDrive/SS.csv')

In [ ]:
#verificacion
df2= pd.DataFrame
df2=df

In [ ]:
df2=df2.drop(columns=['Total1', 'Total2', 'Sector_renombrado', 'ns_labor', 'im_labor','rs_labor'])

In [ ]:
df2= df2.groupby(by='Ciudad').sum()
df2

,Periodo1,Periodo2,Periodo1_0,ns,c_labor
Ciudad,,,,,
ARMENIA,1.335437e+05,124040.442242,1.335437e+05,-0.794090,-9503.292528
AZUAY - CUENCA,2.627590e+05,145648.162469,2.627590e+05,-0.307186,-117110.837531
Abaco,1.114500e+04,13240.000000,1.114500e+04,0.251980,2095.000000
Acapulco,2.942190e+05,283184.000000,2.942190e+05,0.272828,-11035.000000
Aguascalientes,3.739570e+05,419745.000000,3.739570e+05,0.272828,45788.000000
...,...,...,...,...,...
Villahermosa,1.717150e+05,156599.000000,1.717150e+05,0.272828,-15116.000000
Yuma Urbano,1.088184e+06,594105.106666,1.088184e+06,-4.441589,-494078.856484
ZAMORA CHINCHIPE - ZAMORA,1.475100e+04,14298.704653,1.475100e+04,-0.307186,-452.295347


In [ ]:
df2['prueba']= df2['Periodo2']-df2['Periodo1']


In [ ]:
df2.head(20)

,Periodo1,Periodo2,Periodo1_0,ns,c_labor,prueba
Ciudad,,,,,,
ARMENIA,1.335437e+05,1.240404e+05,1.335437e+05,-0.794090,-9503.292528,-9503.292528
AZUAY - CUENCA,2.627590e+05,1.456482e+05,2.627590e+05,-0.307186,-117110.837531,-117110.837531
Abaco,1.114500e+04,1.324000e+04,1.114500e+04,0.251980,2095.000000,2095.000000
Acapulco,2.942190e+05,2.831840e+05,2.942190e+05,0.272828,-11035.000000,-11035.000000
Aguascalientes,3.739570e+05,4.197450e+05,3.739570e+05,0.272828,45788.000000,45788.000000
"Ahuachapán, Santa Ana y Sonsonate.",3.034960e+05,6.335257e+05,3.034960e+05,10.207004,330029.737728,330029.737728
Alto Paraná,3.795440e+05,4.185530e+05,3.795440e+05,0.522642,39009.000000,39009.000000
Antofagasta - Antofagasta,4.349017e+04,7.033707e+04,4.349017e+04,1.545583,26846.891386,26846.891386
Araucanía - Temuco,2.472605e+04,2.322307e+04,2.472705e+04,1.373851,-1502.980452,-1502.980452
